<a href="https://colab.research.google.com/github/MJMortensonWarwick/AI-DL/blob/main/5_2_fine_tuning_a_hugging_face_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning a Huggingface Transformer
In this Notebook we will download a transformer model (based on BERT), fine tune it on a new dataset (the Yelp reviews dataset). We will need to classify the data between one and five stars. We start with some imports:

In [ ]:
!pip install transformers datasets

In [3]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset['train'][0]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/13c31a618ba62568ec8572a222a283dfc29a6517776a3ac5945fb508877dde43. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

Now weill tokenise the data using BERT embeddings

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

  0%|          | 0/650 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

We only need a smaller set of this data for the demo so will take 1,000 random examples for training and the same for validation (no test this time as its just an example and its painfully slow):

In [5]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
val_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

We can now conver this into the Keras format:

In [6]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [7]:
tf_train_dataset = train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = val_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

We will use a general purpose Huggingface model to classify our data:

In [8]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Finally we can (fine-)train the model in the usual fashion. We will keep the learning rate very low (fine tuning) and will only run for 3 epochs (as its slow):

In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

Epoch 1/3
125/125 [==============================] - 345s 3s/step - loss: 0.8563 - sparse_categorical_accuracy: 0.6960 - val_loss: 1.0272 - val_sparse_categorical_accuracy: 0.5560
Epoch 2/3
125/125 [==============================] - 321s 3s/step - loss: 0.8215 - sparse_categorical_accuracy: 0.7280 - val_loss: 1.0185 - val_sparse_categorical_accuracy: 0.5880
Epoch 3/3
125/125 [==============================] - 321s 3s/step - loss: 0.7805 - sparse_categorical_accuracy: 0.7610 - val_loss: 1.0094 - val_sparse_categorical_accuracy: 0.5760


Its a good start and we certainly should be able to see with a few more epochs we would get this to a good quality model. And that's the module! 👊